# Lightweight Hybrid: BFCD with Bilinear Entanglement (High Reps) + Local Search

This notebook implements the **Lightweight Hybrid** approach for portfolio optimization:
- **Ansatz**: BFCD with bilinear entanglement + Local Search post-processing
- **Parameters**: High reps (3-4) + Local Search optimization
- **Expected**: Fast quantum + classical refinement → May reach target faster due to hybrid approach

## Setup and Imports

In [1]:
import os
import sys
import time
import numpy as np
import pickle
from pathlib import Path
from dataclasses import asdict
import docplex.mp.model_reader

# add root directory to path
ROOT = Path(os.getcwd()).parent
sys.path.append(str(ROOT))

# quantum imports
from qiskit_aer import AerSimulator

# project imports
# 1. LP/QUBO processing
from src.sbo.src.utils.lp_utils import load_quadratic_program
from src.sbo.src._converters.quadratic_program_to_qubo import QuadraticProgramToQubo
# 2. ansatz building
from src.step_1 import build_ansatz
# 3. quantum optimization
from src.sbo.src.patterns.building_blocks.step_3 import HardwareExecutor
# 4. local search
from src.step_1 import model_to_obj # same as importing model_to_obj_sparse_numba
from src.sbo.src.optimizer.local_search import repeated_local_search_general
# 5. experiment saving
from src.experiment import Experiment

print(f"Working directory: {ROOT}")
print(f"Python path includes: {str(ROOT)} in sys.path")

Working directory: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main
Python path includes: C:\Users\ASUS\Downloads\WISER_Optimization_VG-main in sys.path


## Problem Setup: 31-Bond Portfolio Optimization

The problem starts as an LP file, which is converted into a quadratic function and then transformed into a Quadratic Unconstrained Binary Optimization (QUBO) problem using predefined modules.

In [2]:
# load the 31-bond problem
lp_file_path = str(ROOT) + '/data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp'

# convert LP file into quadratic problem
quadratic_problem = load_quadratic_program(lp_file_path)

# convert quadratic program into QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(quadratic_problem)

print(f"31-bond problem: {quadratic_problem.get_num_binary_vars()} binary variables")
print(f"QUBO variables: {qubo.get_num_vars()}")
print(f"QUBO objective sense: {qubo.objective.sense}")
print(f"QUBO constant term: {qubo.objective.constant}")

31-bond problem: 31 binary variables
QUBO variables: 31
QUBO objective sense: ObjSense.MINIMIZE
QUBO constant term: 0.0


## Objective Function

The objective function is given by the formula:

$$
\min \sum_{\ell \in L} \sum_{j \in J} \rho_j \left( \sum_{c \in K_\ell} \beta_{c,j} \, x_c - K_{\ell,j}^{\text{target}} \right)^2
$$

The goal is to minimize the difference between a portfolio's characteristics and their target values, ensuring the portfolio matches the targets as closely as possible. This objective function is handled internally by the QUBO formulation.

In [3]:
# create objective function
def objective_function(x):
    """Objective function for the QUBO problem"""
    return qubo.objective.evaluate(x)

## Backend Simulator

**AerSimulator** is a concrete implementation of **BackendV2**, which is the abstract interface/protocol for all quantum backends in Qiskit.

In [4]:
# create backend simulator (AerSimulator)
num_vars = qubo.get_num_vars()
aer_options = {'method': 'matrix_product_state', 'n_qubits': num_vars}
backend = AerSimulator(**aer_options)

print(f"Backend: {backend}")
print(f"Number of qubits: {num_vars}")

Backend: AerSimulator('aer_simulator_matrix_product_state')
Number of qubits: 31


## Ansatz Configuration

In [5]:
# BFCD configuration
ansatz = 'bfcd'
ansatz_params = {
    'reps': 3,  # high reps for deep circuit exploration
    'entanglement': 'bilinear',  # BFCD default (most efficient)
}

print(f"Configuration: {ansatz} with {ansatz_params}")
print(f"Expected: Parameter-efficient with deep circuits")

Configuration: bfcd with {'reps': 3, 'entanglement': 'bilinear'}
Expected: Parameter-efficient with deep circuits


## Ansatz Building

In [6]:
# build BFCD ansatz
ansatz_circuit, initial_layout = build_ansatz(
    ansatz=ansatz,
    ansatz_params=ansatz_params,
    num_qubits=num_vars,
    backend=backend
)

print(f"BFCD Ansatz has {ansatz_circuit.num_parameters} parameters")
print(f"Circuit depth: {ansatz_circuit.depth()}")
print(f"Number of gates: {len(ansatz_circuit.data)}")
print(f"Entanglement: {ansatz_params['entanglement']}")
print(f"Repetitions: {ansatz_params['reps']}")

BFCD Ansatz has 273 parameters
Circuit depth: 34
Number of gates: 665
Entanglement: bilinear
Repetitions: 3


## Conditional Value at Risk (CVaR)

CVaR is a generalization of both the sample mean and the minimum. It represents the expected value of the lowest α-fraction (i.e., the lower tail) of a distribution. In this case, the distribution of sampled energy values from a quantum Hamiltonian.

According to Barkoutsos et al. (2020), to compute CVaR:

- The Hamiltonian is sampled (measured indirectly) *K* times.
- Each sample yields an energy value.
- The energy values are sorted in ascending order (lowest to highest).
- CVaR is then calculated as the average of the lowest ⌈αK⌉ values using the formula:

$$
\text{CVaR}_\alpha = \frac{1}{\lceil \alpha K \rceil} \sum_{k=1}^{\lceil \alpha K \rceil} H_k
$$

Here, $H_k$ denotes the k-th lowest observed energy value.

In implementation:

- The CVaR calculation is performed by the function <code>calc_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.
- Sampled measurement results are mapped to energy values by the function <code>_sampler_result_to_cvar()</code> in <code>"src/sbo/src/patterns/building_blocks/step3.py"</code>.

## Standard VQE

After translating a problem into QUBO form and constructing a Hamiltonian for an n-qubit system, the standard VQE algorithm samples (i.e., indirectly measures) the Hamiltonian *K* times. These measurement outcomes are summed and averaged to estimate the expectation value $\langle \psi(\theta) | H | \psi(\theta) \rangle$. The lowest expectation value, corresponding to a specific set of parameters in the chosen ansatz that minimizes the cost function, is taken as the optimal solution (bitstring) with high probability.

## CVaR-VQE

In contrast, CVaR-VQE optimizes a CVaR objective, focusing on the average of the worst $\alpha$-fraction of sampled energies. The minimal CVaR value, corresponding to certain ansatz parameters minimizing this objective, is considered the optimal solution (bitstring).

## Hardware Executor

The HardwareExecutor orchestrates the entire VQA process. It initializes quantum circuit parameters using π/3 (≈1.047) as a reference value, which provides a good starting point for optimization. The CVaR parameters are set to focus on the worst 10% of measurements (alpha=0.1) with 1024 quantum measurements per iteration.

The executor uses the NFT (Natural Frequency Tuning) optimizer with the following configuration:
- **objective_fun**: QUBO objective function to minimize
- **backend**: Quantum simulator backend
- **isa_ansatz**: BFCD quantum circuit ansatz
- **optimizer_theta0**: Initial parameter values
- **optimizer_method**: NFT optimizer
- **sampler_options**: Quantum measurement settings
- **solver_options**: Maximum 5 iterations (reduced for hybrid), CVaR parameter, random parameter updates

In [7]:
# initialize quantum circuit parameters
theta_initial = np.pi/3 * np.ones(ansatz_circuit.num_parameters)

# cvar optimization parameters
alpha = 0.1   # cvar parameter: focus on worst 10% of measurements
shots = 1024  # number of quantum measurements per iteration

# create HardwareExecutor for quantum optimization (reduced epochs for hybrid)
he = HardwareExecutor(
    objective_fun=objective_function,
    backend=backend,
    isa_ansatz=ansatz_circuit,
    optimizer_theta0=theta_initial,
    optimizer_method="nft",
    refvalue=None,
    sampler_options={'default_shots': shots},
    use_session=False,
    verbose="progress",
    file_name=None,
    store_all_x=True,
    solver_options={
        "max_epoch": 5,  # reduced from 10 to 5 for hybrid approach
        "alpha": alpha,
        "random_update": True
    }
)

## Simulation

The simulation executes the BFCD VQA optimization with CVaR followed by local search refinement. The process involves:

1. Initialization: Quantum circuit parameters are set using π/3 as reference values
2. Iteration Loop: For each optimization iteration:
   - Prepare quantum state using BFCD ansatz
   - Measure Hamiltonian expectation value K times (shots)
   - Calculate CVaR from the worst α-fraction of measurements
   - Update parameters using NFT optimizer
3. Convergence: Continue until convergence or max_epoch limit
4. Local Search Phase: Apply classical local search to refine the best quantum solution
5. Final Solution: Extract the best solution from the hybrid approach

The optimization extracts the best solution found, which includes:
- **best_x**: Binary vector representing selected bonds (1=selected, 0=not selected)
- **best_fx**: Objective function value (CVaR) of the best solution

In [8]:
# run the bfcd VQA optimization with cvar
print("Starting BFCD VQA optimization with CVaR...")

# record start time for performance measurement
start_time = time.time()

# execute the variational quantum optimization
result = he.run()

# record end time and calculate total optimization time
end_time = time.time()
print(f"Quantum optimization completed in {end_time - start_time:.2f} seconds")

# extract the best solution found during quantum optimization
best_x_quantum = he.optimization_monitor.objective_monitor.best_x
best_fx_quantum = he.optimization_monitor.objective_monitor.best_fx

print(f"Quantum best solution: {best_x_quantum}")
print(f"Quantum best objective value: {best_fx_quantum}")

2025-10-05 20:24:46,690 INFO optimization_wrapper: run...
2025-10-05 20:24:46,696 DEBUG optimization_wrapper: Using X0 [1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.0471975

Starting BFCD VQA optimization with CVaR...


2025-10-05 20:24:49,749 INFO optimization_monitor: optimizer internal status: () {'idx_set': array([250, 151, 236, 256, 229, 152, 266,  75, 257,  39, 144, 260, 139,
       271,  43, 187, 142, 143,  77, 169,  33,   1,   0, 195, 262,  81,
       176, 148, 114, 192, 230, 165,  49, 216,  88, 179,  63,  80,  98,
        16, 115,  83,  73,  42, 202, 197,  12,  18, 120, 164, 221, 167,
       159,  61,  10,   5, 127, 209, 190, 137, 181,  79,  60, 163, 228,
       183,   7,  57, 232, 198,  29,  89,  13, 225, 100, 109, 101,  62,
        55,  44, 214, 134,  20, 189,  26, 171,  41, 110, 270,  72, 255,
       129,  11, 241, 245, 246,  47, 175, 265,  87,  78,  31, 126, 200,
        59, 177, 243,  17,   6,  91, 185, 155,  86,  84, 102, 147, 146,
        74, 244, 219, 140, 122,  48,  30, 156, 149, 252, 272,  90,  51,
       222,  54,  65, 210, 253,  37,  96, 138, 106, 191, 111,  24, 104,
        92, 173,  66, 105, 212, 203,  64, 224,   8, 112, 154, 269,  71,
        58,  14, 160, 136, 128,   9,  76, 1

Quantum optimization completed in 7834.85 seconds
Quantum best solution: [0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 1.]
Quantum best objective value: -2237.083089035839


## Post-Processing: Local Search

The local search phase applies classical optimization to refine the best quantum solution.

## Local Search Configuration

The `repeated_local_search_general()` function takes these parameters:
- **x**: Quantum best solution
- **val**: Quantum best objective value
- **func**: Numba-compatible objective function
- **options**: Configuration dictionary

The first two parameters are already obtained, while the remaining two must be set prior to running local search.

### Func: Numba-Compatible Objective Function

Since local search uses numba-compiled code for speed, the CPLEX-parsable LP file must be converted into a docplex model. This model is then passed to the `model_to_obj()` function to generate a numba-compatible objective function. The previously loaded `quadratic_problem` (created from the same LP file) cannot be used because it produces a `QuadraticProgram` object, which is not compatible with the `model_to_obj()` function, as it expects a docplex model.
- LP file → QuadraticProgram → objective function is **not numba-compatible** → usable for quantum phase, **not usable for local search**
- LP file → docplex model → objective function is **numba-compatible** → **usable for local search**

In [9]:
# create a docplex Model object
docplex_model = docplex.mp.model_reader.ModelReader.read(filename)

# decorate the objective function to be numba compatible
numba_compatiable_objective_function = model_to_obj(docplex_model)

31 15


### Options: Configuration Dictionary

- **num_bitflips**: Number of bits to flip per iteration (set to 1 for fine-tuning).
- **maxepoch**: Maximum number of local search epochs (set to 5). This complements the 5 epochs from the quantum phase, resulting in a total of 10 epochs in the hybrid approach.
- **maxfevals_per_variable**: Maximum number of times the objective function is evaluated per variable during one epoch of local search (set to 3 for a balance between solution quality and speed). For example, with 20 variables and this parameter set to 3, the algorithm performs up to 60 evaluations per epoch.
- **refval**: Must be a float (not None) to avoid typing errors. Local search stops when the current value is close to or better than this reference value (set to infinity to disable early stopping).

In [10]:
# create doe_localsearch pattern
doe_ls = {
    'local_search_doe': 'balance',
    'local_search_num_bitflips': 1,
    'local_search_maxiter': None,
    'local_search_maxepoch': 5,
    'local_search_maxfevals_per_variable': 3
}

# calculate maxfevals based on number of variables
if 'local_search_maxfevals_per_variable' in doe_ls:
    doe_ls['local_search_maxfevals'] = len(best_x_quantum) * doe_ls.pop('local_search_maxfevals_per_variable')

# define refval # cannot be none
refval = float('inf')

## Local Search

In [11]:
# apply local search post-processing
print("Starting local search refinement...")

# record start time for performance measurement
local_search_start = time.time()

# call local search
best_x_hybrid, best_fx_hybrid, num_epochs, num_fevals, vals = repeated_local_search_general(
    x=best_x_quantum,
    val=best_fx_quantum,
    func=numba_compatiable_objective_function,
    options=doe_ls | {'refval': refval}
)

# create result dictionary to match postprocess format
refined_solution = {
    'status': 'success',
    'objective': float(best_fx_hybrid),
    'solution': best_x_hybrid,
    'execution_callback_count': he.optimization_monitor.callback_count,
    'execution_results': he.optimization_monitor.list_callback_res,
    'execution_f_value_best': he.optimization_monitor.list_callback_monitor_best,
    'message': "Local search completed successfully"
}

# record end time and calculate total local search time
local_search_end = time.time()
print(f"Local search completed in {local_search_end - local_search_start:.2f} seconds")
print(f"Hybrid best solution: {best_x_hybrid}")
print(f"Hybrid best objective value: {best_fx_hybrid}")
print(f"Improvement: {best_fx_quantum - best_fx_hybrid:.6f}")

Starting local search refinement...
Local search completed in 13.62 seconds
Hybrid best solution: [0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 1.]
Hybrid best objective value: -2237.083089035839
Improvement: 0.000000


## Result

The experiment is saved as a pickle file containing all the necessary data for analysis, reproducibility, debugging, and research. The saved data includes optimization results, convergence data, performance metrics, configuration settings, and experiment metadata.

The experiment object includes:
- Configuration: Ansatz, parameters, optimizer settings, CVaR alpha, local search settings
- Performance: Execution time, iterations, convergence behavior
- Results: Best solution, objective value, relative gap
- Monitoring: Parameter evolution, objective function calls, optimization trajectory
- Hardware: Backend, shots, classical hardware specs

In [12]:
# set path
experiment_path = Path(str(ROOT)+"/project/exp_lightweight_hybrid.pkl")

In [13]:
# create an Experiment object from the hybrid results
# use from_step3()
experiment = Experiment.from_step3(
    experiment_id="lightweight_hybrid_run",
    ansatz="bfcd",
    ansatz_params={'reps': 3, 'entanglement': 'bilinear'},
    theta_initial="piby3",
    device="AerSimulator",
    optimizer="nft",
    alpha=alpha,
    theta_threshold=None,
    lp_file="31bonds/docplex-bin-avgonly-nocplexvars.lp",
    shots=shots,
    refx=None,      # set this for a reference solution
    refvalue=None,  # set this for a reference value
    classical_hw=Experiment.get_current_classical_hw(),
    step3_time=end_time - start_time,
    step3_job_ids=he.job_ids,
    result=result,
    optimization_monitor=he.optimization_monitor
)

# override step4 fields that were set to None in from_step3()
experiment.step4_time = local_search_end - local_search_start
experiment.step4_result_best_x = best_x_hybrid
experiment.step4_result_best_fx = best_fx_hybrid
experiment.step4_iter_best_fx = refined_solution.get('execution_f_value_best', [])

# save the experiment for analysis
experiment_path.parent.mkdir(parents=True, exist_ok=True)
with open(experiment_path, 'wb') as f:
    pickle.dump(asdict(experiment), f)